<a href="https://colab.research.google.com/github/marquesgabi/Doutorado/blob/master/ANN_CLASSIFICA_julho_23_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import zipfile
#import random
from random import randint
from PIL import Image
import re
from sklearn.model_selection import train_test_split
#import scikit-image
import skimage
import pandas as pd

In [2]:
# Import the 'transform' module from 'skimage'
from skimage import transform 

In [3]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# First step: get the csv file (segmented and cropped photos)

In [4]:
!git clone https://github.com/ucfilho/marquesgabi_Julho_2020_using
%cd marquesgabi_Julho_2020_using
Transfere='FotosTreino882_and_Segm.zip'
file_name = zipfile.ZipFile(Transfere, 'r')
file_name.extractall()

fatal: destination path 'marquesgabi_Julho_2020_using' already exists and is not an empty directory.
/content/marquesgabi_Julho_2020_using


In [5]:
labels = [] #name files

with zipfile.ZipFile(Transfere, "r") as f:
  for f in f.namelist():
    labels.append(f)

In [6]:
df=pd.read_csv(labels[0])


In [7]:
Num=len(labels)
df=pd.read_csv(labels[0])
for i in range(1,Num):
  df_new=pd.read_csv(labels[i])
  frames = [df, df_new]
  df= pd.concat(frames, ignore_index=True)

y_valor=df['Type']

In [8]:
print(df.tail())

     Unnamed: 0 Type          0  ...        782        783  Width
985           7    B   84.33883  ...    7.14600    7.38484  142.0
986           8    I  104.73698  ...    2.05457    0.38150  163.0
987           9    I   75.70248  ...  107.38844  106.00001  154.0
988          10    I   56.64465  ...    3.15316    0.40107  187.0
989          11    B  111.50165  ...    0.00000    0.00000  194.0

[5 rows x 787 columns]


In [9]:
print(df.head())

   Unnamed: 0 Type        0        1  ...      781      782      783  Width
0           0    G  0.00000  0.00000  ...  0.68642  0.68631  0.68769    NaN
1           1    G  0.00000  0.00000  ...  0.47277  0.49262  0.50909    NaN
2           2    G  0.45321  0.52257  ...  0.54645  0.54919  0.55439    NaN
3           3    G  0.75367  0.64590  ...  0.43356  0.48524  0.61876    NaN
4           4    G  0.94521  0.89736  ...  0.98942  0.98108  0.95838    NaN

[5 rows x 787 columns]


In [10]:
#print(df)

In [11]:
df_teste=pd.read_csv(labels[2])
print(df_teste.head())

   Unnamed: 0  Width Type          0  ...       780       781        782        783
0           0    157    I   71.90012  ...  49.57118  35.95513    7.75954    1.03862
1           1    124    I  127.06763  ...  94.57544  98.71695  100.52861  100.84391
2           2    182    G   87.10651  ...   5.33136   5.84024    5.89941    5.65089
3           3    101    I   99.01176  ...  35.15087  41.39006   43.76875   44.09146
4           4    183    I   56.30844  ...  52.96656  53.65711   53.44656   53.11703

[5 rows x 787 columns]


In [12]:
print(df.tail())

     Unnamed: 0 Type          0  ...        782        783  Width
985           7    B   84.33883  ...    7.14600    7.38484  142.0
986           8    I  104.73698  ...    2.05457    0.38150  163.0
987           9    I   75.70248  ...  107.38844  106.00001  154.0
988          10    I   56.64465  ...    3.15316    0.40107  187.0
989          11    B  111.50165  ...    0.00000    0.00000  194.0

[5 rows x 787 columns]


In [13]:
Fotos=df.drop(['Unnamed: 0','Type','Width'], axis=1)

In [14]:
print(np.array(Fotos).shape )

(990, 784)


In [15]:
print(Fotos)

             0          1          2  ...        781        782        783
0      0.00000    0.00000    0.00315  ...    0.68642    0.68631    0.68769
1      0.00000    0.00000    0.00000  ...    0.47277    0.49262    0.50909
2      0.45321    0.52257    0.56642  ...    0.54645    0.54919    0.55439
3      0.75367    0.64590    0.62497  ...    0.43356    0.48524    0.61876
4      0.94521    0.89736    0.61852  ...    0.98942    0.98108    0.95838
..         ...        ...        ...  ...        ...        ...        ...
985   84.33883   93.31998  103.44733  ...    7.52371    7.14600    7.38484
986  104.73698  108.48595  108.84282  ...    7.58692    2.05457    0.38150
987   75.70248   76.11571   76.23968  ...  106.21488  107.38844  106.00001
988   56.64465   56.24413   54.42472  ...   11.69516    3.15316    0.40107
989  111.50165  117.65936  105.70792  ...    0.00000    0.00000    0.00000

[990 rows x 784 columns]


# Second step: create the ANN to evaluate which is a grain in photos segmented


In [16]:
images28=np.array(df)

In [17]:
print(y_valor)

0      G
1      G
2      G
3      G
4      G
      ..
985    B
986    I
987    I
988    I
989    B
Name: Type, Length: 990, dtype: object


In [18]:
#for i in labels:
#  print(i)
#different_type=['B','V','N','I']

img_name=y_valor

Graos='G' # numero 1
Indef='I' # numero 2
Buraco='B' # numero 3
Varios='V' # numero 4
NaoGrao='N' # numero 5
y_new_valor=[]
y_G=[]; y_N=[]; y_I=[]; y_B=[]; y_V=[]

for x in img_name:
  if re.search(Graos, x):
    y_new_valor.append(1)
    y_G.append(x)
  elif re.search(Indef, x):
    y_new_valor.append(2)
    y_I.append(x)
  elif re.search(Buraco, x):
    y_new_valor.append(3)
    y_G.append(x)
  elif re.search(Varios, x):
    y_new_valor.append(4)
    y_V.append(x)
  else:
    y_new_valor.append(5)
    y_N.append(x)

print(y_N)
Num=len(y_N);print(Num) 

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
115


In [19]:
print (y_new_valor)
print(len(y_new_valor))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 4, 4, 4, 4, 4, 4, 3, 3, 3, 

In [20]:
#Define data train and data test
W_train, W_test, yw_train, yw_test = train_test_split(np.array(Fotos), np.array(y_new_valor), 
                                                    test_size=0.30, 
                                                    random_state=42)

In [21]:
train_images=W_train #imagens utilizadas para o treino
train_labels=yw_train # resposta da rede
test_images=W_test
test_labels=yw_test

In [22]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10)
])

In [23]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [24]:
model.fit(train_images, train_labels, epochs=10)

Epoch 1/10
22/22 [==============================] - 0s 2ms/step - loss: 5.6770 - accuracy: 0.2670
Epoch 2/10
22/22 [==============================] - 0s 2ms/step - loss: 2.9020 - accuracy: 0.3074
Epoch 3/10
22/22 [==============================] - 0s 2ms/step - loss: 2.4552 - accuracy: 0.3434
Epoch 4/10
22/22 [==============================] - 0s 2ms/step - loss: 1.7069 - accuracy: 0.3983
Epoch 5/10
22/22 [==============================] - 0s 2ms/step - loss: 1.4189 - accuracy: 0.4170
Epoch 6/10
22/22 [==============================] - 0s 3ms/step - loss: 1.3703 - accuracy: 0.4574
Epoch 7/10
22/22 [==============================] - 0s 2ms/step - loss: 1.7005 - accuracy: 0.3752
Epoch 8/10
22/22 [==============================] - 0s 2ms/step - loss: 1.4478 - accuracy: 0.4661
Epoch 9/10
22/22 [==============================] - 0s 2ms/step - loss: 1.3121 - accuracy: 0.5137
Epoch 10/10
22/22 [==============================] - 0s 2ms/step - loss: 1.3298 - accuracy: 0.4921


In [ ]:
#arrumar com o bira?????
data = {'y_Actual': y_valor,
        'y_Predicted': prediction
        }

df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
#print (df)

In [ ]:
#arrumar com o bira?????
data = {'y_Predicted': prediction,
        'y_Actual': y_valor
        }

df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
print (df)